# Postgres Database connector
## How to connect to Postgres Database
### 1. I have my own Postgres service
If you already have a running Postgres database service you can connect to it using your credentials, the same you use to connect from pgAdmin or psql.

### 2. Using Bodo.ai test docker image for PyDough
You can run Pydough with our pre-built Postgres test image available on Docker Hub:

Make sure you have Docker or Podman installed.

Pull and run the image with the following command:

```shell
  export POSTGRES_USER=[USER_NAME]
  export POSTGRES_PASSWORD=[PASSWORD]
  export POSTGRES_DB="pydough-test"
  export POSTGRES_HOST="localhost"

  docker run -d --name [CONTAINER_NAME] \
    -e POSTGRES_USER=${POSTGRES_USER} \
    -e POSTGRES_PASSWORD=${POSTGRES_PASSWORD} \
    -e POSTGRES_DB=${POSTGRES_DB} \
    -p 5432:5432 \
    docker.io/juankxbodo/pydough-pgsql-tests:latest
```
Replace [CONTAINER_NAME] with your preferred container name. If not defined on `docker run`, the `pydough-pgsql-tests` container will run with the following default values:

```shell
POSTGRES_USER=test_user
POSTGRES_PASSWORD=test_pass
POSTGRES_DB=pydough-test
```

## Installing Postgres Connector
To connect from PyDough to a Postgres service you need to have psycopg2 already installed:

`uv pip install psycopg2-binary`

## Setting connection parameters and connecting to Postgres

In [ ]:
# Import libraries
import pydough

# Set postgreSQL connection parameters
postgres_user: str = "test_user"
postgres_password: str = "test_pass"
postgres_db: str = "pydough-test"
postgres_host: str = "localhost"

# Metadata setup
pydough.active_session.load_metadata_graph("tests/test_metadata/defog_graphs.json", "Broker")

# Setup postgreSQL connection
pydough.active_session.connect_database("postgres", user=postgres_user,password=postgres_password,dbname=postgres_db,host=postgres_host)


## Enabling PyDough's Jupyter extension
When you first setup the notebook you need to load the extension with `%load_ext pydough.jupyter_extensions`. 

It will allow you to:
- Write PyDough in notebook cells using %%pydough refering directly to Pydough collections.
- Automatically render results

To know what Pydough elements are available to be used you can print the knowledge graph structure using `pydough.explain_structure(graph)`

In [ ]:
%load_ext pydough.jupyter_extensions
graph = pydough.active_session.metadata
print(pydough.explain_structure(graph))

## Running PyDough querys in Postgres

We can run the following PyDough query in broker database to answer: "How many transactions were made by customers from the USA last week (exclusive of the current week)? Return the number of transactions and total transaction amount".



In [ ]:
%%pydough
# Define the start of the current week, assuming the week starts on Monday.
start_of_current_week = DATETIME('now', 'start of week')

# Define the start of the previous week by subtracting 7 days from the start of the current week.
start_of_last_week = DATETIME(start_of_current_week, '-7 days')

# Filter transactions to include only those from customers in the USA that occurred last week.
# The date range is inclusive of the start of last week and exclusive of the start of the current week.
last_week_usa_transactions = transactions.WHERE(
(LOWER(customer.country) == 'usa') &
(date_time >= start_of_last_week) &
(date_time < start_of_current_week)
)

# Calculate the final aggregates: the count of the filtered transactions and the sum of their amounts.
result = Broker.CALCULATE(
num_transactions=COUNT(last_week_usa_transactions),
total_transaction_amount=SUM(last_week_usa_transactions.amount)
)

pydough.to_df(result)